# **Time Series**

In [1]:
pip install openeo

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 30.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 40.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╺━━━━━━ 5/6 [openeo]]  WARNING: The script openeo-auth is installed in '/usr/local/python/3.12.1/bin' which is not on PATH.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6/6 [openeo]

[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import openeo
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

In [3]:
connection = openeo.connect("openeo.dataspace.copernicus.eu").authenticate_oidc()

Visit https://identity.dataspace.copernicus.eu/auth/realms/CDSE/device?user_code=SPCL-BIGQ 📋 to authenticate.

✅ Authorized successfully

Authenticated using device code flow.


In [4]:
aoi = {
    "type": "Polygon",
    "coordinates": [
        [
            [112.7836159632455, -7.044797153824376],  # Titik 1 (Barat Laut / North-West)
            [112.7836159632455, -7.0464931472063626], # Titik 2 (Barat Daya / South-West)
            [112.7854009999511, -7.0464931472063626], # Titik 3 (Tenggara / South-East)
            [112.7854009999511, -7.044797153824376],  # Titik 4 (Timur Laut / North-East)
            [112.7836159632455, -7.044797153824376],  # Kembali ke Titik 1 
        ]
    ],
}

In [5]:
s5no2 = connection.load_collection(
    "SENTINEL_5P_L2",
    temporal_extent=["2021-01-01", "2023-12-31"],
    spatial_extent={
        "west": 112.7836159632455,
        "south": -7.0464931472063626,
        "east": 112.7854009999511,
        "north": -7.044797153824376,
    },
    bands=["NO2"],
)


In [6]:
# Menggabungkan (meng-aggregate) semua data dalam satu hari menjadi rata-rata harian (daily mean)
s5no2 = s5no2.aggregate_temporal_period(reducer="mean", period="day")

# Melakukan agregasi spasial terhadap area poligon(aoi)
s5no2 = s5no2.aggregate_spatial(reducer="mean", geometries=aoi)
